In [1]:
# Authors: Samantha Tetef and Vikram Kashyap
# Date: Winter 2020-2021 to Summer 2021

In [2]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../moldl/')
import moldl

import importlib
from pathlib import Path

importlib.reload(moldl)
from moldl import *

In [9]:
def PhosphorusSorter(mol):
    p = mol.getAtoms(element=15)[0]  # get phosphorus atom
    bonds = mol.getBonds(atom=p)  # get all bonds to P atom
    doubleo, singleo, ohgroup, other = 0,0,0,0  # functional/ R group counters
    for b in bonds:
        partner = b.partnerof(p)
        if partner.element == 8:  # if oxygen
            if b.order == 2: doubleo +=1
            if b.order == 1:
                singleo +=1
                for obond in mol.getBonds(atom=partner):
                    if obond.partnerof(partner).element==1:
                        ohgroup+=1
                        singleo-=1
        else:
            other +=1
    if (doubleo, singleo, ohgroup, other) == (1,3,0,0):
        return 'phosphate_esters'
    elif (doubleo, singleo, ohgroup, other) == (1,0,2,1):
        return 'phosphonic_acids'
    elif (doubleo, singleo, ohgroup, other) == (1,2,0,1):
        return 'phosphonate'
    elif (doubleo, singleo, ohgroup, other) == (0,3,0,0):
        return 'phosphite_esters'
    elif (doubleo, singleo, ohgroup, other) == (1,0,1,2):
        return 'phosphenic_acids'
    elif (doubleo, singleo, ohgroup, other) == (1,1,0,2):
        return 'phosphinate'
    elif (doubleo, singleo, ohgroup, other) == (1,0,0,3):
        return 'phosphine_oxides'
    elif (doubleo, singleo, ohgroup, other) == (0,0,0,3):
        return 'trialkyl_phosphines'
    elif (doubleo, singleo, ohgroup, other) == (0,0,0,4):
        return 'tetraalkyl_phosphonium'
    elif (doubleo, singleo, ohgroup, other) == (0,0,0,2):
        return 'phosphaalkenes'
    else:
        return 'other'

In [10]:
db = MolDB('Database/')

with Path('processed_cids.list').open() as f:
    processed_cids = f.read().splitlines()
processed_cids = set(processed_cids)

# add to moldl database using the processsed spectrum cids
iterator = 0
for cid in processed_cids:
    if cid not in db.molecules:
        db.addMolecule(Molecule.from_cid(cid))
        print(f'{iterator}\r', end="")
        iterator += 1

In [11]:
sortedids = db.sort(PhosphorusSorter, processed_cids)
# sortedids['phosphonic acids']

listdir = Path('Categories/')
for category in sortedids:
    with (listdir/(str(category)+'.list')).open('w') as f:
        for cid in sortedids[category]:
            f.write(str(cid)+'\n')